In [255]:
pwd()

"/home/zyu20/git/BulkLMM.jl/test"

In [2]:
cd("..")

In [3]:
Threads.nthreads()

24

In [4]:
include("BXDdata_for_test.jl"); ## Load BXD data used for testing:

In [5]:
using LoopVectorization

In [6]:
include("../src/scan_LiteQTL.jl"); ## Load multithreaded functions to be tested:

In [7]:
include("../src/parallel_helpers.jl");

In [8]:
BLAS.set_num_threads(24)

In [9]:
BLAS.get_num_threads()

24

In [367]:
hsq_list = collect(0:1000) * 0.001;

In [368]:
kinship_Gc = CSV.read("run-gemma/output/kinship.cXX.txt", DataFrame, delim = '\t', header = false) |> Matrix;

In [369]:
@time bulkscan_LODs = bulkscan(pheno, geno, kinship_Gc, hsq_list);

LoadError: InterruptException:

In [370]:
bulkscan_LODs[1:6, :]

6×35556 Matrix{Float64}:
 0.122362  0.123282  0.111683  0.0568243  …  0.0473728  0.355458  0.550632
 0.122362  0.123282  0.111683  0.0568243     0.0473728  0.355458  0.550632
 0.122362  0.123282  0.111683  0.0568243     0.0473728  0.355458  0.550632
 0.122362  0.123282  0.111683  0.0568243     0.0473728  0.355458  0.550632
 0.122362  0.123282  0.111683  0.0568243     0.0473728  0.355458  0.550632
 0.122362  0.123282  0.111683  0.0568243  …  0.0473728  0.355458  0.550632

In [371]:
gemma = "/home/zyu20/Softwares/gemma-0.98.5-linux-static-AMD64";

In [372]:
gemma_results = readdlm("run-gemma/output/gemma_results.txt.assoc.txt", '\t');

In [373]:
gemma_results

7322×10 Matrix{Any}:
   "chr"  "rs"             "ps"  …    "logl_H1"   "l_mle"   "p_lrt"
 -9       "rs31443144"   -9         68.5713      0.441307  0.13839
 -9       "rs6269442"    -9         68.5713      0.441307  0.13839
 -9       "rs32285189"   -9         68.5713      0.441307  0.13839
 -9       "rs258367496"  -9         68.5713      0.441307  0.13839
 -9       "rs32430919"   -9      …  68.5713      0.441307  0.13839
 -9       "rs36251697"   -9         68.5713      0.441307  0.13839
 -9       "rs30658298"   -9         68.5713      0.441307  0.13839
 -9       "rs51852623"   -9         68.5713      0.441307  0.13839
 -9       "rs31879829"   -9         68.5713      0.441307  0.13839
 -9       "rs36742481"   -9      …  68.5713      0.441307  0.13839
 -9       "rs6365999"    -9         68.5713      0.441307  0.13839
 -9       "rs13470446"   -9         68.5713      0.441308  0.138394
  ⋮                              ⋱                         
 -9       "rs31466210"   -9      …  67.5696   

In [374]:
gemma_results[7272, :]

10-element Vector{Any}:
 -9
   "rs259771792"
 -9
  0
   "A"
   "B"
  0.468
 67.49315
  1.0e6
  0.8424859

In [375]:
function p2lod(pval::Float64, df::Int64)
    
    lrs = invlogcdf(Chisq(df), log(1-pval))
    lod = lrs/(2*log(10))
    
    # return lrs
    return lod

end

p2lod (generic function with 1 method)

In [376]:
traitID = 20001;

In [377]:
gemma_LODs = map(x -> p2lod(x, 1), gemma_results[2:end, 10]);

In [378]:
hcat(gemma_LODs, bulkscan_LODs[:, traitID])

7321×2 Matrix{Float64}:
 0.476805   0.524201
 0.476805   0.524201
 0.476805   0.524201
 0.476805   0.524201
 0.476805   0.524201
 0.476805   0.524201
 0.476805   0.524201
 0.476805   0.524201
 0.476805   0.524201
 0.476805   0.524201
 0.476805   0.524201
 0.476796   0.524193
 0.220659   0.255113
 ⋮          
 0.041786   0.0828429
 0.041786   0.0828429
 0.041786   0.0828429
 0.0912584  0.258224
 0.207958   0.392154
 0.207958   0.392154
 0.0334485  0.221615
 0.187448   0.372084
 0.170059   0.355096
 0.348018   0.528934
 0.167993   0.372557
 0.167993   0.372557

In [379]:
scan_null_lod = scan_null(reshape(pheno[:, traitID], :, 1), geno, kinship_Gc; reml = true);

In [380]:
scan_alt_lod = scan_alt(reshape(pheno[:, traitID], :, 1), geno, kinship_Gc; reml = false);

In [381]:
scan_null_lod[1:2]

(0.01114455751028617, 0.2864436579121599)

In [382]:
scan_alt_lod[1:2]

(0.011113871349546575, 0.3316961451332981)

In [383]:
hcat(gemma_LODs, scan_null_lod[3], scan_alt_lod[3], bulkscan_LODs[:, traitID])

7321×4 Matrix{Float64}:
 0.476805   0.468774   0.476804   0.524201
 0.476805   0.468774   0.476804   0.524201
 0.476805   0.468774   0.476804   0.524201
 0.476805   0.468774   0.476804   0.524201
 0.476805   0.468774   0.476804   0.524201
 0.476805   0.468774   0.476804   0.524201
 0.476805   0.468774   0.476804   0.524201
 0.476805   0.468774   0.476804   0.524201
 0.476805   0.468774   0.476804   0.524201
 0.476805   0.468774   0.476804   0.524201
 0.476805   0.468774   0.476804   0.524201
 0.476796   0.468766   0.476796   0.524193
 0.220659   0.220129   0.220659   0.255113
 ⋮                                
 0.041786   0.0413815  0.041786   0.0828429
 0.041786   0.0413815  0.041786   0.0828429
 0.041786   0.0413815  0.041786   0.0828429
 0.0912584  0.0873108  0.0912585  0.258224
 0.207958   0.0573483  0.0641354  0.392154
 0.207958   0.0573483  0.0641354  0.392154
 0.0334485  0.011237   0.0139704  0.221615
 0.187448   0.0197437  0.0255083  0.372084
 0.170059   0.0187493  0.0241257  0

In [384]:
maxSqDiff(reshape(gemma_LODs, :, 1), reshape(bulkscan_LODs[:, traitID], :, 1))

0.1616233398037279

In [385]:
sumSqDiff(reshape(gemma_LODs, :, 1), reshape(bulkscan_LODs[:, traitID], :, 1))

133.24042084150756

In [386]:
diff = gemma_LODs .- bulkscan_LODs[:, traitID];

In [387]:
sqDiff = diff.^2

7321-element Vector{Float64}:
 0.0022464670231387473
 0.0022464670231387473
 0.002246467023138379
 0.002246467023138379
 0.0022464670231387473
 0.0022464670231387473
 0.002246467023138379
 0.002246467023138379
 0.002246467023138379
 0.002246467023138379
 0.0022464669453011892
 0.0022464033013353825
 0.0011870968467121682
 ⋮
 0.0016856686609860836
 0.0016856686935232909
 0.0016856686935232909
 0.027877344900074784
 0.033928442213412166
 0.033928442213412166
 0.035406502229482884
 0.03409044954198345
 0.034238815069574
 0.032730552125557685
 0.04184605246470777
 0.04184605246470777